# Milestone 1

## Purpose:
In this notebook, we will attempt to download a data dump containing daily rainfall over NSW, Australia dataset found on [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681). This data dump is comprised of different files contained in a 776.4 MB compressed format. This size of the uncompressed data dump is approximately 6.6 GB. 

In [1]:
# Imports
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## Creating the API request

We will use the [requests library](https://docs.python-requests.org/en/master/) to generate an http request call to the [figshare API](https://docs.figshare.com/). Specifically, we will make a call to the `/articles` endpoint to retrieve information about the article of interest including the url we need to download the data.

In [7]:
article_id = 14096681
base_url = 'https://api.figshare.com/v2'
headers = {"Content-Type": "application/json"}
endpoint = f'/articles/{article_id}'
output_directory = "rainfall/"

In [9]:
response = requests.request("GET", base_url+endpoint, headers=headers)
data = json.loads(response.text)
data

{'defined_type_name': 'dataset',
 'embargo_date': None,
 'citation': 'Beuzen, Tomas (2021): Daily rainfall over NSW, Australia. figshare. Dataset. https://doi.org/10.6084/m9.figshare.14096681.v3',
 'url_private_api': 'https://api.figshare.com/v2/account/articles/14096681',
 'embargo_reason': '',
 'references': ['https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6',
  'https://pangeo-data.github.io/pangeo-cmip6-cloud/',
  'https://www.longpaddock.qld.gov.au/silo/'],
 'funding_list': [],
 'url_public_api': 'https://api.figshare.com/v2/articles/14096681',
 'id': 14096681,
 'custom_fields': [],
 'size': 814109773,
 'metadata_reason': '',
 'funding': None,
 'figshare_url': 'https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681',
 'embargo_type': 'file',
 'title': 'Daily rainfall over NSW, Australia',
 'defined_type': 3,
 'embargo_options': [],
 'is_embargoed': False,
 'version': 3,
 'resource_doi': None,
 'url_public_html': 'https://figshare.com/articles/dataset/Dai

The response above contains a `data` json key which is of interest of us. It lists the files corresponding to the article along with their download urls.

In [17]:
files = data["files"]           
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

The data dump is `data.zip` which can be retreived as follows

In [14]:
%%time
files_to_dl = ["data.zip"] 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 6.11 s, sys: 4.54 s, total: 10.7 s
Wall time: 2min 9s


As seen in the output above, the single act of downloading the data dump to the local machine took around 2mins 9s.

## Comparison

Comparison of running time for the download operation across 4 different machines

|           | CPU               | RAM   | HD         | Operation time |
|-----------|-------------------|-------|------------|----------------|
| Machine 1 | i5-4460 @ 3.20Ghz | 10 GB | 1 TB SSD | 2 min 9 sec   |
| Machine 2 |                   |       |            |                |
| Machine 3 |                   |       |            |                |
| Machine 4 |                   |       |            |                |

## Extracting the data

Here, we attempt to extract the data dump we downloaded `data.zip` into individual uncompressed csv files.

In [18]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 16.7 s, sys: 2.28 s, total: 19 s
Wall time: 19.5 s


## Comparison

Comparison of running time for the extraction operation across 4 different machines

|           | CPU               | RAM   | HD         | Operation time |
|-----------|-------------------|-------|------------|----------------|
| Machine 1 | i5-4460 @ 3.20Ghz | 10 GB | 1 TB SSD | 19.5 sec   |
| Machine 2 |                   |       |            |                |
| Machine 3 |                   |       |            |                |
| Machine 4 |                   |       |            |                |

## Combining the data

### Pandas Approach

Here, we attempt to combine the individuals csv files into a single pandas dataframe which we then save to a single csv file called `combined_data.csv`. We create a new column called `model` to be able to identify which dataset each record originally comes from. The names of the models are extracted using regex from the csv file names.

In [20]:
%%time
%memit
files = glob.glob('rainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\\]+(?=\_d)', file)[0])
                for file in files)
              )
df.to_csv("rainfall/combined_data.csv")

peak memory: 183.35 MiB, increment: 0.30 MiB
CPU times: user 5min 43s, sys: 12 s, total: 5min 55s
Wall time: 5min 58s


Check the output

In [3]:
%%sh
du -sh rainfall/combined_data.csv

6.1G	rainfall/combined_data.csv


The size of the combined dataset is 6.1 GB!

In [ ]:
%%time
import pandas as pd
df = pd.read_csv("rainfall/combined_data.csv")

In [ ]:
print(df.shape)

In [ ]:
df.head()

Attempting to read the above csv file results in a dead kernel in jupyterlab. System Resources Monitor shows a steady increase in RAM usage until 100% after which the jupyterlab notebook crashes as seen in the image below.
![out-of-memory](img/machine1-1.png)

## Comparison

Results of the concatenation operation across 4 different machines

|           | CPU               | RAM   | HD         | Operation time |
|-----------|-------------------|-------|------------|----------------|
| Machine 1 | i5-4460 @ 3.20Ghz | 10 GB | 1 TB SSD | OUT OF MEMORY ERROR  |
| Machine 2 |                   |       |            |                |
| Machine 3 |                   |       |            |                |
| Machine 4 |                   |       |            |                |

### DASK Approach

In [5]:
#// TO DO